In [1]:
import pandas as pd
from scripts.data_split.stratifiedSplit import stratified_split
from scripts.TreatImbalance.BalancingTrainingData import hybrid_balance
from scripts.Training.TuneEvaluate import tune_and_evaluate 
from sklearn.tree import DecisionTreeClassifier


In [2]:
data_path = "../data/preprocessed/preprocessed_reduced_data.csv"

target_col = "fire"
test_size = 0.2
desired_minority_prop = (
        0.30  # user-chosen: 0.30 means 30% minority in balanced training set
    )
balanced_train_savepath = "balanced_train.csv"


data_df = pd.read_csv(data_path)
print("Loaded dataset with shape:", data_df.shape)
if target_col not in data_df.columns:
        raise ValueError(
            f"Target column '{target_col}' not found in CSV columns: {data_df.columns.tolist()}"
        )


Loaded dataset with shape: (735483, 41)


In [3]:
train_df, test_df = stratified_split(
        data_df, target_col=target_col, test_size=test_size, random_state=42)

In [ ]:
balanced_train_df = hybrid_balance(
        train_df,
        target_col=target_col,
        minority_target=1,
        desired_minority_prop=desired_minority_prop,
        random_state=42,
        save_path=balanced_train_savepath,
        verbose=True,
    )


Training size: 588386 (minority=49559, majority=538827)
Desired minority proportion: 0.30
Undersampling majority from 538827 -> 411870 (fast reduction).


In [4]:
balanced_train_df = pd.read_csv(balanced_train_savepath)
    # 3) tune decision tree with cross-validation on balanced training set and evaluate on untouched test set
dt = DecisionTreeClassifier(random_state=42)

dt_grid = {
    "criterion": ["entropy"],
    "max_depth": [8, 12, 20],
    "min_samples_split": [ 5, 10],
    "min_samples_leaf": [5, 10],
}

results_dt = tune_and_evaluate(
    balanced_train_df,
    test_df,
    estimator=dt,
    param_grid=dt_grid,
    scoring="recall",
)



🚀 Starting GridSearchCV...


=== Total combinations: 12 ===

🔧 Running combination 1/12
Params: {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 5}
-----------------------------------
📌 Cross-validation metrics:
   • Recall:    0.8808  ± 0.0028
   • Accuracy:  0.9698  ± 0.0007
   • Precision: 0.9993  ± 0.0003
   • F1-score:  0.9363  ± 0.0016
   • ROC AUC:   0.9688  ± 0.0011
-----------------------------------

🔧 Running combination 2/12
Params: {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 5, 'min_samples_split': 10}
-----------------------------------
📌 Cross-validation metrics:
   • Recall:    0.8808  ± 0.0028
   • Accuracy:  0.9698  ± 0.0007
   • Precision: 0.9993  ± 0.0003
   • F1-score:  0.9363  ± 0.0016
   • ROC AUC:   0.9688  ± 0.0011
-----------------------------------

🔧 Running combination 3/12
Params: {'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 10, 'min_samples_split': 5}
-----------------------------------

In [5]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn_grid = {
    "n_neighbors": [ 5, 7, 9 ],
    "weights": ["uniform"],
    "p": [2],     # 1 = Manhattan, 2 = Euclidean
}

results_knn = tune_and_evaluate(
    balanced_train_df,
    test_df,
    estimator=knn,
    param_grid=knn_grid,
    scoring="recall",   
)



🚀 Starting GridSearchCV...


=== Total combinations: 3 ===

🔧 Running combination 1/3
Params: {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
-----------------------------------
📌 Cross-validation metrics:
   • Recall:    0.9515  ± 0.0006
   • Accuracy:  0.9851  ± 0.0002
   • Precision: 0.9888  ± 0.0004
   • F1-score:  0.9698  ± 0.0005
   • ROC AUC:   0.9909  ± 0.0003
-----------------------------------

🔧 Running combination 2/3
Params: {'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}
-----------------------------------
📌 Cross-validation metrics:
   • Recall:    0.9461  ± 0.0002
   • Accuracy:  0.9843  ± 0.0002
   • Precision: 0.9909  ± 0.0008
   • F1-score:  0.9680  ± 0.0004
   • ROC AUC:   0.9921  ± 0.0004
-----------------------------------

🔧 Running combination 3/3
Params: {'n_neighbors': 9, 'p': 2, 'weights': 'uniform'}
-----------------------------------
📌 Cross-validation metrics:
   • Recall:    0.9421  ± 0.0003
   • Accuracy:  0.9836  ± 0.0003
   • Precision: 0.9924  ± 0.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

rf_grid = {
    "n_estimators": [100, 300],
    "max_depth": [10, 20, 30],
    "min_samples_split": [ 5, 10],
    "min_samples_leaf": [5, 10],
    "class_weight": [None],
}

results_rf = tune_and_evaluate(
    balanced_train_df,
    test_df,
    estimator=rf,
    param_grid=rf_grid,
    scoring="recall",   # best for imbalanced classification
)



🚀 Starting GridSearchCV...


=== Total combinations: 24 ===

🔧 Running combination 1/24
Params: {'class_weight': None, 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 100}
-----------------------------------
📌 Cross-validation metrics:
   • Recall:    0.8861  ± 0.0009
   • Accuracy:  0.9712  ± 0.0002
   • Precision: 0.9994  ± 0.0001
   • F1-score:  0.9393  ± 0.0005
   • ROC AUC:   0.9903  ± 0.0003
-----------------------------------

🔧 Running combination 2/24
Params: {'class_weight': None, 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 300}
-----------------------------------
📌 Cross-validation metrics:
   • Recall:    0.8859  ± 0.0005
   • Accuracy:  0.9712  ± 0.0001
   • Precision: 0.9994  ± 0.0001
   • F1-score:  0.9392  ± 0.0003
   • ROC AUC:   0.9907  ± 0.0003
-----------------------------------

🔧 Running combination 3/24
Params: {'class_weight': None, 'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 1